In [17]:
# Code credits: Adapted bits and pieces from https://github.com/webdataset/webdataset/blob/master/docs/gettingstarted.ipynb

import sys
sys.path.append('..')

import gc
import json
import os
from itertools import islice
from datetime import datetime
import pytz
from pytz import timezone
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import skimage.transform as st
import tqdm

import torch
import torch.optim as optim
from torchvision import transforms
import webdataset as wds

from model.selfattn_3d_cnn import *
from model.baseline_3d_cnn import *
from model.resattn_3d_cnn import *
from utils.model_utils import *
from utils.model_run import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_dir = '../data'
shards_dir = os.path.join(data_dir, 'shards_new')

# Opening JSON file
with open('../parameters.json') as json_file:
    parameters = json.load(json_file)

batch_size = parameters['batch_size']
shard_size = parameters['shard_size']
parameters

{'batch_size': 4, 'shard_size': 16}

In [3]:
urls = [os.path.join(shards_dir, it) for it in os.listdir(shards_dir) if it.endswith('.tar')]

# Try to overfit on smaller data
# urls = urls[:round(len(urls)*0.3)]

# Another shard directory, continued; realize can't use because keys will collide cuz we refreshed...
# shards_dir2 = os.path.join(data_dir, 'shards_new_cont')
# urls += [os.path.join(shards_dir2, it) for it in os.listdir(shards_dir2) if it.endswith('.tar')]


# All the data
total_num_shards = round(len(urls)*0.75)
train_urls = urls[:round(total_num_shards*0.7)]
val_urls = urls[round(total_num_shards*0.7):round(total_num_shards*0.85)]
test_urls = urls[round(total_num_shards*0.85):]

# Smaller data just to run model once
# train_urls = urls[:2]
# val_urls = urls[2:3]
# test_urls = urls[3:]


print("Number of train shards:", len(train_urls))
print("Number of validation shards:", len(val_urls))
print("Number of test shards:", len(test_urls))

Number of train shards: 50
Number of validation shards: 11
Number of test shards: 35


In [4]:
# Create dataset objects
train_iternum = len(train_urls)*shard_size//batch_size
val_iternum = len(val_urls)*shard_size//batch_size
test_iternum = len(test_urls)*shard_size//batch_size

print("Number of iterations per train epoch:", train_iternum)

train_dataset = (
    wds
    .WebDataset(train_urls, length=train_iternum)
    .shuffle(shard_size)
    .decode('torch')
    .to_tuple('volumes.pyd', 'labels.pyd', 'studynames.pyd')
    .batched(batch_size)
#     .map_tuple(pre_transforms, identity, identity)
)
loader_train = torch.utils.data.DataLoader(train_dataset, num_workers=0, batch_size=None) #setting batch_size = None disables batching

val_dataset = (
    wds
    .WebDataset(val_urls, length=val_iternum)
    .shuffle(shard_size)
    .decode('torch')
    .to_tuple('volumes.pyd', 'labels.pyd', 'studynames.pyd')
    .batched(batch_size)
)
loader_val = torch.utils.data.DataLoader(val_dataset, num_workers=0, batch_size=None)

test_dataset = (
    wds
    .WebDataset(test_urls, length=test_iternum)
    .shuffle(shard_size)
    .decode('torch')
    .to_tuple('volumes.pyd', 'labels.pyd', 'studynames.pyd')
    .batched(batch_size)
)
loader_test = torch.utils.data.DataLoader(test_dataset, num_workers=0, batch_size=None)

# for image, target in islice(dataset, 0, 2):
#     print(image.shape)

Number of iterations per train epoch: 200


In [5]:
gc.collect()

66

In [6]:
USE_GPU = True
dtype = torch.float

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
#     dtype = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')

print(device)
print(dtype)

cuda
torch.float32


In [410]:
# Make log directory and checkpoint directory (DIFFERENT DIRECTORY FROM BASELINE)
dir_nm = datetime.now(tz=pytz.utc).astimezone(timezone('US/Pacific')).strftime('%Y-%m-%d_%H-%M-%S')
# dir_nm = "first_mini_c2fc2"
log_dir = os.path.join('../runs/baseline', dir_nm) # running from this notebook since the other one gives cuda memory errors
# log_dir = os.path.join('../runs/experiment', dir_nm)
os.mkdir(log_dir)
os.mkdir(os.path.join(log_dir, 'Checkpoints'))


# Model, optimizer, criterion
model = baseline_3DCNN(in_num_ch=1)
# model = selfattn_3DCNN(in_num_ch=1)
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
criterion = torch.nn.BCEWithLogitsLoss()

In [7]:
gc.collect()

44

In [412]:
# Baseline model
train_loss_dict, val_loss_dict = train(model, optimizer, criterion, loader_train, loader_val, log_dir, device=device, epochs=10, val_every=5)

Epoch 1:   2%|▎         | 5/200 [02:46<2:56:10, 54.21s/batch, loss=0.704]

Total iteration 5, validation loss = 0.7028



Epoch 1:   5%|▌         | 10/200 [05:19<2:47:13, 52.81s/batch, loss=0.708]

Total iteration 10, validation loss = 0.7026



Epoch 1:   8%|▊         | 15/200 [07:52<2:41:49, 52.49s/batch, loss=0.628]

...

Total iteration 1999, validation loss = 0.5581



Epoch 10:  98%|█████████▊| 195/200 [1:32:19<04:02, 48.57s/batch, loss=0.501]

Total iteration 2004, validation loss = 0.5571



Epoch 10: 100%|██████████| 200/200 [1:34:29<00:00, 28.35s/batch, loss=0.32] 

Total iteration 2009, validation loss = 0.5541



In [413]:
torch.save(model.state_dict(), '../runs/baseline/baseline_final_model.pt')

In [414]:
# Common errors and how to fix them:

# Error:
#   RuntimeError: running_mean should contain 1 elements not 8
# Fix: One of your batchnorm 3D parameter values is off

# Error:
#   RuntimeError: CUDA out of memory.
# Fix: Make the model / batch size smaller 
# First try to make batch size smaller. Will require longer training time possibly but does not decrease expressivity of model.
# If need to decrease complexity of model, 

# Error:
#   RuntimeError: Given groups=1, weight of size [1, 1, 1, 1, 1], expected input[8, 4, 5, 32, 32] to have 1 channels, but got 4 channels instead
# Fix: Wrong number of in_channels in self attention layer

# Experiment 1: 3D Self-Attention after 3D Conv Layers

In [9]:
# Make log directory and checkpoint directory (DIFFERENT DIRECTORY FROM BASELINE)
dir_nm = datetime.now(tz=pytz.utc).astimezone(timezone('US/Pacific')).strftime('%Y-%m-%d_%H-%M-%S')
# dir_nm = "first_mini_c2fc2"
# log_dir = os.path.join('../runs/baseline', dir_nm) # running from this notebook since the other one gives cuda memory errors
log_dir = os.path.join('../runs/experiment_att', dir_nm)
os.mkdir(log_dir)
os.mkdir(os.path.join(log_dir, 'Checkpoints'))


# Model, optimizer, criterion
# model = baseline_3DCNN(in_num_ch=1)
model2 = selfattn_3DCNN(in_num_ch=1)
optimizer2 = optim.Adam(model2.parameters(), lr = 1e-4)
criterion2 = torch.nn.BCEWithLogitsLoss()

In [10]:
# Experimental model
train_loss_dict2, val_loss_dict2 = train(model2, optimizer2, criterion2, loader_train, loader_val, log_dir, device=device, epochs=10, val_every=5)

Epoch 1:   2%|▏         | 4/200 [00:28<16:18,  4.99s/batch, loss=0.669]

Total iteration 5, validation loss = 0.6871


Epoch 1:   2%|▎         | 5/200 [02:46<2:55:06, 53.88s/batch, loss=0.671]

Epoch 1:   4%|▍         | 9/200 [03:03<44:29, 13.98s/batch, loss=0.675]  

Total iteration 10, validation loss = 0.6864


...

Total iteration 2004, validation loss = 0.5410


Epoch 10:  98%|█████████▊| 195/200 [1:32:54<03:55, 47.19s/batch, loss=0.43]

Epoch 10:  99%|█████████▉| 198/200 [1:32:58<00:33, 16.99s/batch, loss=0.428]/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
Epoch 10: 100%|█████████▉| 199/200 [1:32:59<00:12, 12.13s/batch, loss=0.129]

Total iteration 2009, validation loss = 0.5382


Epoch 10: 100%|██████████| 200/200 [1:35:04<00:00, 28.52s/batch, loss=0.377]

In [11]:
torch.save(model2.state_dict(), '../runs/experiment_att/experiment_final_model.pt')

In [ ]:
# a = torch.randn(4, 40, 512, 512)
# print(a.size())
# transforms.Resize(size=(256, 256))(a).size()

# Experiment 2: Residual/Skip Connection after Self-Attention

In [17]:
# Make log directory and checkpoint directory (DIFFERENT DIRECTORY FROM BASELINE AND EXPERIMENT 1)
dir_nm = datetime.now(tz=pytz.utc).astimezone(timezone('US/Pacific')).strftime('%Y-%m-%d_%H-%M-%S')
# dir_nm = "first_mini_c2fc2"
# log_dir = os.path.join('../runs/baseline', dir_nm) # running from this notebook since the other one gives cuda memory errors
log_dir = os.path.join('../runs/experiment_res', dir_nm)
os.mkdir(log_dir)
os.mkdir(os.path.join(log_dir, 'Checkpoints'))


# Model, optimizer, criterion
# model = baseline_3DCNN(in_num_ch=1)
model3 = resattn_3DCNN(in_num_ch=1)
optimizer3 = optim.Adam(model3.parameters(), lr = 1e-4)
criterion3 = torch.nn.BCEWithLogitsLoss()

In [18]:
# Experimental model
train_loss_dict3, val_loss_dict3 = train(model3, optimizer3, criterion3, loader_train, loader_val, log_dir, device=device, epochs=10, val_every=5)

Epoch 1:   2%|▏         | 4/200 [00:20<11:57,  3.66s/batch, loss=0.698]

Total iteration 5, validation loss = 0.6742


Epoch 1:   2%|▎         | 5/200 [02:04<2:11:56, 40.60s/batch, loss=0.754]

Epoch 1:   4%|▍         | 9/200 [02:17<34:20, 10.79s/batch, loss=0.722]  

Total iteration 10, validation loss = 0.6685


...

Total iteration 2004, validation loss = 0.5727


Epoch 10:  98%|█████████▊| 195/200 [1:14:34<03:25, 41.04s/batch, loss=0.627]

Epoch 10: 100%|█████████▉| 199/200 [1:14:38<00:10, 10.60s/batch, loss=0.746]

Total iteration 2009, validation loss = 0.5746


Epoch 10: 100%|██████████| 200/200 [1:16:19<00:00, 22.90s/batch, loss=0.508]

In [19]:
torch.save(model3.state_dict(), '../runs/experiment_res/experimentres_final_model.pt')

### Test models on test set:

In [7]:
models = []

In [8]:
ckpt_dir = os.path.join('..', 'runs', 'baseline')
ckpt_path = os.path.join(ckpt_dir, 'baseline_final_model.pt') # COLAB
ckpt = torch.load(ckpt_path)

ckpt_model = baseline_3DCNN(in_num_ch=1)
ckpt_model.load_state_dict(ckpt)

models.append(ckpt_model)

In [9]:
ckpt_dir = os.path.join('..', 'runs', 'experiment_att')
ckpt_path = os.path.join(ckpt_dir, 'experiment_final_model.pt') 
ckpt = torch.load(ckpt_path)

ckpt_model = selfattn_3DCNN(in_num_ch=1)
ckpt_model.load_state_dict(ckpt)

models.append(ckpt_model)

In [10]:
ckpt_dir = os.path.join('..', 'runs', 'experiment_res')
ckpt_path = os.path.join(ckpt_dir, 'experimentres_final_model.pt') 
ckpt = torch.load(ckpt_path)

ckpt_model = resattn_3DCNN(in_num_ch=1)
ckpt_model.load_state_dict(ckpt)

models.append(ckpt_model)


In [15]:
model_names = ['baseline', 'experiment_att', 'experiment_res']
criterion = torch.nn.BCEWithLogitsLoss()
gc.collect()

88

In [20]:
test_losses = []
for modname, mod in zip(model_names, models):
    print("Testing start for model:", modname)
    
    log_dir = os.path.join('..', 'runs', modname+'_test')
    writer = SummaryWriter(log_dir)
    mod.eval()
    
    test_loss = test_model(loader_test, mod, criterion, writer, device)
    test_losses.append(test_loss)

  0%|          | 0/140 [00:00<?, ?batch/s]

Testing start for model: baseline


  0%|          | 0/140 [00:00<?, ?batch/s]

Test loss = 0.5364
Testing start for model: experiment_att


  0%|          | 0/140 [00:00<?, ?batch/s]

Test loss = 0.5188
Testing start for model: experiment_res


100%|██████████| 140/140 [05:18<00:00,  2.27s/batch]

Test loss = 0.5729
